In [ ]:
# importing dependencies
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import gmaps
import os

# import openweather API key
from Config import g_key

In [ ]:
#importing the csv from part1
path = "../Resources/CityWeather.csv"
Vacation_DF = pd.read_csv(path, low_memory = False)

In [ ]:
#putting api keys and locations&humidity
gmaps.configure(api_key=g_key)
locations = Vacation_DF[["Latitude","Longitude"]].astype(float)
humidity = Vacation_DF["Humidity"].astype(float)

In [ ]:
# heatmaps for google
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)

fig

In [ ]:
# now that the heatmap is working, we need to narrow down the dataframe for MY ideal weather condition for vacation.

Hotel_df = Vacation_DF.loc[ (Vacation_DF["Max Temp"]<77) & (Vacation_DF["Max Temp"]>70) & (Vacation_DF["Wind Speed"]<8)
                           & (Vacation_DF["Cloudiness"]==0)]


In [ ]:
Hotel_df.head(15)

In [ ]:
#creating an empty column for the hotel name
Hotel_df["Hotel Name"] = ""

In [ ]:
#setting up google parameters:
parameters = {"radius":5000, "types":"hotel", "keyword":"hotel", "key":g_key}

In [ ]:
# using locations to get the nearest hotels

for index, x in Hotel_df.iterrows():
    lat = x["Latitude"]
    long = x["Longitude"]
    
    parameters["location"] = f"{lat},{long}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_request = requests.get(url, params=parameters).json()
    
    try:
        Hotel_df.loc[index, "Hotel Name"] = hotel_request["results"][0]["name"]
    except (KeyError, IndexError):
        print("No Result. Skipped!!")



In [ ]:
#im not sure why, but the dropna does not work. so we can use iloc instead
Hotel_df = Hotel_df.sort_values("Hotel Name")
Hotel_df = Hotel_df.iloc[3:16,:]

In [ ]:
Hotel_df

In [ ]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
hotel_locations = Hotel_df[["Latitude", "Longitude"]]

In [ ]:
#now we can add the hotel marker on top of the heatmap. yes!!!!

hotel_layer = gmaps.marker_layer(hotel_locations, info_box_content = hotel_info )

fig.add_layer(hotel_layer)

fig